In [1]:
# @title ⚙️🧱 CELL 0 — Environment & CSV Knowledge Base Setup (Production-Ready)
# English-only comments as requested.

import os
import sys
import logging
import torch
import warnings
import importlib.util
import subprocess
import shutil
from typing import List
import pandas as pd
from glob import glob

print("🚀 Initializing Enhanced CELL 0 Environment Setup…")

# ================================================================================
# 1) GPU Hardware Validation
# ================================================================================
print("\n--- 1. Checking GPU Hardware ---")
if not torch.cuda.is_available():
    warnings.warn("CUDA not detected — GPU acceleration unavailable.")
    print("🔥 WARNING: No GPU detected. Performance will be slow.")
else:
    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA version: {torch.version.cuda}")

# ================================================================================
# 2) Python Dependency Validation
# ================================================================================
print("\n--- 2. Validating Python Dependencies ---")

required_packages = {
    "transformers": "transformers",
    "accelerate": "accelerate",
    "sentencepiece": "sentencepiece",

    "langchain": "langchain",
    "langchain-community": "langchain_community",
    "langchain-huggingface": "langchain_huggingface",
    "chromadb": "chromadb",

    "sentence-transformers": "sentence_transformers",
    "tiktoken": "tiktoken",

    "pandas": "pandas",
    "pyarrow": "pyarrow",
    "tqdm": "tqdm",
}

missing_pkgs = []
for pip_name, import_name in required_packages.items():
    if importlib.util.find_spec(import_name) is None:
        missing_pkgs.append(pip_name)
        print(f"⚠️ Missing: {import_name} → installing ({pip_name})")

if missing_pkgs:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "--upgrade"] + missing_pkgs)
    print("✅ Installed missing packages.")
else:
    print("✅ All required packages already installed.")

# ================================================================================
# 3) System Dependencies (ffmpeg for Whisper)
# ================================================================================
print("\n--- 3. Checking system dependencies (ffmpeg) ---")
if shutil.which("ffmpeg") is None:
    print("⚠️ ffmpeg NOT FOUND — audio transcription will fail.")
else:
    print("✅ ffmpeg detected.")

# ================================================================================
# 4) Project Config (Upgraded)
# ================================================================================
class Config:
    # HF Models
    EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
    LLM_MODEL_NAME = "ALLaM-AI/ALLaM-7B-Instruct-preview"
    ASR_MODEL_NAME = "openai/whisper-large-v3"

    # RAG Engine Settings
    CHUNK_SIZE = 1500
    CHUNK_OVERLAP = 250
    MEMORY_WINDOW = 4
    SERVICES_PER_DISPLAY_PAGE = 5
    ARABIC_NORMALIZATION = True
    ENABLE_QUERY_REWRITER = True
    ENABLE_ENGLISH_TRANSLATION = True

    # Absolute CSV paths
    DATA_MASTER_DIR = "/ibex/user/rashidah/projects/MOI_ChatBot/chatbot_project/1_data/Data_Master"
    DATA_CHUNKS_DIR = "/ibex/user/rashidah/projects/MOI_ChatBot/chatbot_project/1_data/Data_chunks"

    # Project Folder Structure
    PROJECT_DIR = "chatbot_project"
    DATA_DIR = os.path.join(PROJECT_DIR, "1_data")
    PROCESSED_DIR = os.path.join(PROJECT_DIR, "2_processed")
    MODELS_DIR = os.path.join(PROJECT_DIR, "3_models")
    OUTPUTS_DIR = os.path.join(PROJECT_DIR, "4_outputs")
    LOGS_DIR = os.path.join(OUTPUTS_DIR, "logs")
    AUDIO_DIR = os.path.join(OUTPUTS_DIR, "audio")
    VECTOR_DB_DIR = os.path.join(PROJECT_DIR, "vector_db")

    # Logging
    LOG_FILE = os.path.join(LOGS_DIR, "app.log")

# ================================================================================
# 5) Create Required Directories
# ================================================================================
print("\n--- 4. Creating Project Directory Structure ---")

for folder in [
    Config.PROJECT_DIR,
    Config.DATA_DIR,
    Config.PROCESSED_DIR,
    Config.MODELS_DIR,
    Config.OUTPUTS_DIR,
    Config.LOGS_DIR,
    Config.AUDIO_DIR,
    Config.VECTOR_DB_DIR,
]:
    os.makedirs(folder, exist_ok=True)

print(f"✅ Directory structure initialized under: {Config.PROJECT_DIR}")

# ================================================================================
# 6) Logging Initialization
# ================================================================================
print("\n--- 5. Initializing Logger ---")

logging.basicConfig(
    level=logging.INFO,
    filename=Config.LOG_FILE,
    filemode="w",
    format="%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s",
    force=True
)

logging.info("Enhanced CELL 0 initialized successfully.")
print(f"📝 Logging to: {Config.LOG_FILE}")

# ================================================================================
# 7) Validate the CSV Knowledge Base
# ================================================================================
print("\n--- 6. Validating Master + Chunk CSVs ---")

MASTER_SCHEMA = [
    "service_id", "service_title_ar", "description_full",
    "beneficiaries", "fees", "conditions", "access_path",
]

CHUNK_SCHEMA = [
    "chunk_id", "service_id", "chunk_title",
    "chunk_text", "chunk_type", "language", "meta",
]

def ensure_folder(path, name):
    if not os.path.isdir(path):
        raise FileNotFoundError(f"❌ {name} folder not found: {path}")
    print(f"✅ {name} folder detected: {path}")

def list_csv(path):
    return sorted(glob(os.path.join(path, "*.csv")))

def validate_schema(csv_file, schema):
    df = pd.read_csv(csv_file, nrows=2)
    missing = [c for c in schema if c not in df.columns]
    if missing:
        raise ValueError(f"❌ Schema mismatch in {csv_file}: missing {missing}")

ensure_folder(Config.DATA_MASTER_DIR, "Master CSVs")
ensure_folder(Config.DATA_CHUNKS_DIR, "Chunk CSVs")

master_files = list_csv(Config.DATA_MASTER_DIR)
chunks_files = list_csv(Config.DATA_CHUNKS_DIR)

print(f"📦 Master files found: {len(master_files)}")
print(f"📦 Chunk files found:  {len(chunks_files)}")

print("\n🔍 Validating CSV schemas…")
for f in master_files[:3]:
    validate_schema(f, MASTER_SCHEMA)
    print(f"   ✅ OK → {os.path.basename(f)}")

for f in chunks_files[:3]:
    validate_schema(f, CHUNK_SCHEMA)
    print(f"   ✅ OK → {os.path.basename(f)}")

print("\n🎉 CELL 0 COMPLETE — Environment & CSV Knowledge Base Loaded Successfully.\n")


🚀 Initializing Enhanced CELL 0 Environment Setup…

--- 1. Checking GPU Hardware ---
✅ GPU detected: NVIDIA A100-SXM4-80GB
✅ CUDA version: 12.8

--- 2. Validating Python Dependencies ---
✅ All required packages already installed.

--- 3. Checking system dependencies (ffmpeg) ---
✅ ffmpeg detected.

--- 4. Creating Project Directory Structure ---
✅ Directory structure initialized under: chatbot_project

--- 5. Initializing Logger ---
📝 Logging to: chatbot_project/4_outputs/logs/app.log

--- 6. Validating Master + Chunk CSVs ---
✅ Master CSVs folder detected: /ibex/user/rashidah/projects/MOI_ChatBot/chatbot_project/1_data/Data_Master
✅ Chunk CSVs folder detected: /ibex/user/rashidah/projects/MOI_ChatBot/chatbot_project/1_data/Data_chunks
📦 Master files found: 10
📦 Chunk files found:  10

🔍 Validating CSV schemas…
   ✅ OK → ahwal_master.csv
   ✅ OK → amn_master.csv
   ✅ OK → hajj_master.csv
   ✅ OK → ahwal_rag_chunks.csv
   ✅ OK → amn_rag_chunks.csv
   ✅ OK → hajj_rag_chunks.csv

🎉 CELL 0 

In [2]:
# @title ⚙️🧱 CELL 1 — Data Processing for CSV Knowledge Base (Masters + Chunks)
# English-only comments as requested.

import os
import re
import json
import glob
import logging
import unicodedata
from typing import List, Dict, Any
import pandas as pd
from IPython.display import display, HTML
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("📊 Starting CELL 1 — CSV Knowledge Base Processing…")
logging.info("Starting CELL 1 processing pipeline.")

# ======================================================================
# 0) CONFIG PATHS
# ======================================================================
MASTER_DIR = Config.DATA_MASTER_DIR
CHUNKS_DIR = Config.DATA_CHUNKS_DIR
PROCESSED_DIR = Config.PROCESSED_DIR

os.makedirs(PROCESSED_DIR, exist_ok=True)

# ======================================================================
# 1) REQUIRED SCHEMAS
# ======================================================================
MASTER_REQUIRED = [
    "service_id", "service_title_ar", "description_full",
    "beneficiaries", "fees", "conditions", "access_path"
]

CHUNK_REQUIRED = [
    "chunk_id", "service_id", "chunk_title",
    "chunk_text", "chunk_type", "language", "meta"
]

# ======================================================================
# 2) Arabic Normalization + Soft Cleaning
# ======================================================================
ARABIC_DIAC = re.compile(r"[\u0617-\u061A\u064B-\u0652\u0670\u06D6-\u06ED]")

def normalize_arabic(text: str) -> str:
    """Apply consistent Arabic normalization for RAG stability."""
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFC", text)
    text = ARABIC_DIAC.sub("", text)
    text = re.sub(r"[أإآٱ]", "ا", text)
    text = text.replace("ى", "ي")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def soft_clean(text: str) -> str:
    """Remove markdown markers and noisy formatting."""
    if not isinstance(text, str):
        return ""
    text = re.sub(r"\*{1,2}(.*?)\*{1,2}", r"\1", text)
    text = re.sub(r"_+(.*?)_+", r"\1", text)
    text = re.sub(r"#+\s*", "", text)
    text = re.sub(r"\[cite[^\]]*\]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def clean_df(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str).map(soft_clean).map(normalize_arabic)
    return df

# ======================================================================
# 3) Sector Inference
# ======================================================================
SECTOR_MAP = {
    "jawazat": "الجوازات",
    "muroor": "المرور",
    "ahwal": "الأحوال المدنية",
    "waffedeen": "شؤون الوافدين",
    "tafweed": "إدارة التفاويض",
    "prisons": "المديرية العامة للسجون",
    "amn": "الأمن العام",
    "niyaba": "النيابة العامة",
    "moiministry": "وزارة الداخلية",
    "hajj": "وزارة الحج والعمرة"
}

def infer_sector(path: str) -> str:
    base = os.path.basename(path).lower()
    key = base.split("_")[0]
    return SECTOR_MAP.get(key, "غير محدد")

# ======================================================================
# 4) Load CSVs
# ======================================================================
master_files = sorted(glob.glob(os.path.join(MASTER_DIR, "*.csv")))
chunk_files = sorted(glob.glob(os.path.join(CHUNKS_DIR, "*.csv")))

print(f"📦 Master files: {len(master_files)}")
print(f"📦 Chunk files : {len(chunk_files)}")

if not master_files or not chunk_files:
    raise RuntimeError("❌ Missing master/chunk CSV files.")

# ======================================================================
# 5) Validate + Read CSVs
# ======================================================================
def validate(df, schema, name):
    missing = [c for c in schema if c not in df.columns]
    if missing:
        raise ValueError(f"❌ Missing columns in {name}: {missing}")

masters, chunks = [], []

for f in master_files:
    df = pd.read_csv(f)
    validate(df, MASTER_REQUIRED, f)
    df["__sector"] = infer_sector(f)
    df["__source_file"] = f
    masters.append(df)

for f in chunk_files:
    df = pd.read_csv(f)
    validate(df, CHUNK_REQUIRED, f)
    df["__sector"] = infer_sector(f)
    df["__source_file"] = f
    chunks.append(df)

df_master = pd.concat(masters, ignore_index=True)
df_chunks = pd.concat(chunks, ignore_index=True)

print(f"🧾 Master rows: {len(df_master)}")
print(f"🧩 Chunk rows : {len(df_chunks)}")

# ======================================================================
# 6) Normalize all text columns
# ======================================================================
df_master = clean_df(df_master, [
    "service_title_ar","description_full","beneficiaries",
    "fees","conditions","access_path"
])

df_chunks = clean_df(df_chunks, [
    "chunk_title","chunk_text","chunk_type","language","meta"
])

df_chunks = df_chunks[df_chunks["chunk_text"].str.strip() != ""]

# ======================================================================
# 7) Build Master-Level Documents
# ======================================================================
def build_master_text(row):
    parts = [
        f"اسم الخدمة: {row['service_title_ar']}",
        f"الوصف: {row['description_full']}"
    ]
    if row["beneficiaries"]: parts.append(f"المستفيدون: {row['beneficiaries']}")
    if row["fees"]: parts.append(f"الرسوم: {row['fees']}")
    if row["conditions"]: parts.append(f"الشروط: {row['conditions']}")
    if row["access_path"]: parts.append(f"طريقة الوصول: {row['access_path']}")
    return " | ".join(parts)

service_docs_list = []
for (sector, sid), g in df_master.groupby(["__sector", "service_id"]):
    best_row = g.loc[g["description_full"].str.len().idxmax()]
    service_docs_list.append({
        "sector": sector,
        "service_id": sid,
        "service_title_ar": best_row["service_title_ar"],
        "text": build_master_text(best_row),
        "source_files": list(g["__source_file"].unique())
    })

df_service_docs = pd.DataFrame(service_docs_list)
print(f"🧱 Service docs built: {len(df_service_docs)}")

processed_service_documents = [
    Document(
        page_content=r["text"],
        metadata={
            "sector": r["sector"],
            "service_id": r["service_id"],
            "service_title": r["service_title_ar"],
            "doc_level": "service_master",
            "source_files": r["source_files"]
        }
    )
    for _, r in df_service_docs.iterrows()
]

# ======================================================================
# 8) Build Chunk-Level Documents
# ======================================================================
splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=120,
    length_function=len
)

processed_chunks = []

for _, row in df_chunks.iterrows():
    meta = {
        "sector": row["__sector"],
        "service_id": row["service_id"],
        "chunk_id": row["chunk_id"],
        "chunk_title": row["chunk_title"],
        "chunk_type": row["chunk_type"],
        "language": row["language"],
        "meta": row["meta"],
        "source_file": row["__source_file"],
        "doc_level": "service_chunk"
    }

    text = row["chunk_text"]
    if len(text) > 1000:
        parts = splitter.split_text(text)
        for i, p in enumerate(parts):
            m = dict(meta)
            m["chunk_part"] = f"{i+1}/{len(parts)}"
            processed_chunks.append(Document(page_content=p, metadata=m))
    else:
        processed_chunks.append(Document(page_content=text, metadata=meta))

print(f"✂️ Chunk documents prepared: {len(processed_chunks)}")

# ======================================================================
# 9) Save artifacts
# ======================================================================
preview_path = os.path.join(PROCESSED_DIR, "service_docs_preview.jsonl")
with open(preview_path, "w", encoding="utf-8") as f:
    for d in processed_service_documents:
        f.write(json.dumps({"text": d.page_content, "metadata": d.metadata}, ensure_ascii=False) + "\n")

stats_path = os.path.join(PROCESSED_DIR, "chunk_docs_stats.json")
with open(stats_path, "w", encoding="utf-8") as f:
    json.dump({
        "num_service_docs": len(processed_service_documents),
        "num_chunk_docs": len(processed_chunks),
        "master_files": master_files,
        "chunk_files": chunk_files
    }, f, ensure_ascii=False, indent=2)

print(f"💾 Preview saved → {preview_path}")
print(f"💾 Stats saved   → {stats_path}")

display(HTML("<h2 style='text-align:center;color:green;'>✅ CELL 1 completed successfully</h2>"))

print("\n🎉 CELL 1 COMPLETE.\n")


📊 Starting CELL 1 — CSV Knowledge Base Processing…
📦 Master files: 10
📦 Chunk files : 10
🧾 Master rows: 64
🧩 Chunk rows : 256
🧱 Service docs built: 64
✂️ Chunk documents prepared: 256
💾 Preview saved → chatbot_project/2_processed/service_docs_preview.jsonl
💾 Stats saved   → chatbot_project/2_processed/chunk_docs_stats.json



🎉 CELL 1 COMPLETE.



In [3]:
# @title 🧠 CELL 2 — Load AI Models & Build/Load FAISS Vector Store (Pro RAG, Final Version)
# English-only comments as requested.

import os
import shutil
import logging
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

print("🔧 Starting CELL 2 — Pro RAG Model Initialization…")
logging.info("CELL 2 started.")

# ==============================================================================
# 0) Pre-run validation
# ==============================================================================
if "processed_chunks" not in globals() or not processed_chunks:
    raise RuntimeError("❌ processed_chunks missing — run CELL 1 first.")

if not torch.cuda.is_available():
    raise RuntimeError("❌ CUDA GPU not available — A100/V100 required.")

# ==============================================================================
# 1) Initialize globals
# ==============================================================================
embedding_model = None
llm_tokenizer = None
llm_model = None
asr_pipeline = None
vector_store = None
rag_models_and_vector_store_ready = False

VECTOR_DIR = Config.VECTOR_DB_DIR
os.makedirs(VECTOR_DIR, exist_ok=True)

# ==============================================================================
# 2) Helper functions: load/build FAISS
# ==============================================================================
def load_faiss(path, embeddings):
    fbin = os.path.join(path, "index.faiss")
    if not os.path.exists(fbin):
        return None
    print(f"📂 Found existing FAISS index → {path}")

    try:
        vs = FAISS.load_local(
            path,
            embeddings,
            allow_dangerous_deserialization=True
        )
        vs.similarity_search("اختبار", k=1)  # sanity check
        print("✅ Existing FAISS index loaded & verified.")
        return vs

    except Exception as e:
        print(f"❌ FAISS load error: {e}")
        print("🗑 Removing corrupted FAISS directory and rebuilding…")
        shutil.rmtree(path, ignore_errors=True)
        os.makedirs(path, exist_ok=True)
        return None


def build_faiss(docs, embeddings, path):
    print(f"⚡ Building FAISS for {len(docs)} documents… (GPU accelerated)")
    vs = FAISS.from_documents(docs, embedding=embeddings)
    vs.save_local(path)
    print(f"✅ New FAISS index saved → {path}")
    return vs

# ==============================================================================
# 3) Progress bar
# ==============================================================================
pbar = tqdm(total=4, bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} {desc}")

# ==============================================================================
# 4) Step 1 — BGE-M3 Embeddings
# ==============================================================================
pbar.set_description("Step 1: Embeddings (bge-m3)")
print("🔹 Loading BGE-M3 embedding model on GPU…")

embedding_model = HuggingFaceEmbeddings(
    model_name=Config.EMBEDDING_MODEL_NAME,
    cache_folder=Config.MODELS_DIR,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

# Warmup to avoid first-call latency
_ = embedding_model.embed_query("اختبار")

pbar.update(1)

# ==============================================================================
# 5) Step 2 — Load/Build FAISS Vector Store
# ==============================================================================
pbar.set_description("Step 2: Vector Store (FAISS)")

vector_store = load_faiss(VECTOR_DIR, embedding_model)
if vector_store is None:
    vector_store = build_faiss(processed_chunks, embedding_model, VECTOR_DIR)

pbar.update(1)

# ==============================================================================
# 6) Step 3 — Load ALLaM 7B LLM
# ==============================================================================
pbar.set_description("Step 3: LLM (ALLaM 7B)")

os.environ["TRANSFORMERS_CACHE"] = Config.MODELS_DIR

print("🔹 Loading ALLaM 7B (bf16)…")
llm_tokenizer = AutoTokenizer.from_pretrained(
    Config.LLM_MODEL_NAME,
    cache_dir=Config.MODELS_DIR,
    use_fast=False
)

llm_model = AutoModelForCausalLM.from_pretrained(
    Config.LLM_MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=Config.MODELS_DIR,
    low_cpu_mem_usage=True
)

print("✅ LLM ready.")
pbar.update(1)

# ==============================================================================
# 7) Step 4 — Load Whisper Large V3 (No Warmup / No Bytes)
# ==============================================================================
pbar.set_description("Step 4: ASR (Whisper Large V3)")

print("🔹 Loading Whisper Large V3 (FP32)…")

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=Config.ASR_MODEL_NAME,
    device="cuda:0",
    torch_dtype=torch.float32,
    model_kwargs={
        "cache_dir": Config.MODELS_DIR,
        "attn_implementation": "sdpa"
    }
)

# ✅ No warmup needed — avoids ValueError from invalid audio input.

pbar.update(1)
pbar.close()

# ==============================================================================
# 8) Final confirmation
# ==============================================================================
rag_models_and_vector_store_ready = True

print("\n✅✅✅ CELL 2 COMPLETE — Enhanced Models + Vector DB Loaded ✅✅✅")
logging.info("CELL 2 complete.")


🔧 Starting CELL 2 — Pro RAG Model Initialization…


Step 1: Embeddings (bge-m3):   0%|           | 0/4 Step 1: Embeddings (bge-m3): 

🔹 Loading BGE-M3 embedding model on GPU…


Step 2: Vector Store (FAISS):  25%|██▌        | 1/4 Step 2: Vector Store (FAISS): 

📂 Found existing FAISS index → chatbot_project/vector_db


...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Step 4: ASR (Whisper Large V3):  75%|███████▌   | 3/4 Step 4: ASR (Whisper Large V3): 

✅ LLM ready.
🔹 Loading Whisper Large V3 (FP32)…


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
Step 4: ASR (Whisper Large V3): 100%|██████████ | 4/4 Step 4: ASR (Whisper Large V3): 


✅✅✅ CELL 2 COMPLETE — Enhanced Models + Vector DB Loaded ✅✅✅


In [4]:
# @title 🔗 CELL 4 — Pro RAG Chain (Rewrite → Retrieve → Rerank → Answer) [FIXED]
# English-only comments as requested.

import re
import logging
import warnings
from typing import List, Tuple, Dict, Any
from dataclasses import dataclass

from langchain.schema import Document
from langchain_community.retrievers import BM25Retriever
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

warnings.filterwarnings("ignore")

print("🧠 Building PRO RAG chain (v2 — Post-hoc Translation)…")
logging.info("CELL 4 (Fixed) started.")

# ---------------- Safety Checks ----------------
if "rag_models_and_vector_store_ready" not in globals() or not rag_models_and_vector_store_ready:
    raise RuntimeError("❌ CELL 2 not ready.")
if "processed_chunks" not in globals() or not processed_chunks:
    raise RuntimeError("❌ CELL 1 not ready.")

# ---------------- Arabic Normalization ----------------
ARABIC_DIAC = re.compile(r"[\u0617-\u061A\u064B-\u0652\u0670\u06D6-\u06ED]")

def normalize_ar(text: str) -> str:
    if not isinstance(text, str): return ""
    t = ARABIC_DIAC.sub("", text)
    t = re.sub(r"[أإآٱ]", "ا", t)
    t = t.replace("ى", "ي")
    return re.sub(r"\s+", " ", t).strip()

def looks_english(s: str):
    return bool(re.search(r"[A-Za-z]", s))

# ⭐️ NEW HELPER: Check for Arabic text
def is_arabic(text: str) -> bool:
    """Check if the text contains Arabic characters."""
    return bool(re.search(r'[\u0600-\u06FF]', text))

# ---------------- Synonym Expansion ----------------
SYNONYMS = {
    "جواز": ["جواز السفر", "تجديد الجواز", "passport", "renew passport"],
    "هوية": ["الهوية الوطنية", "بطاقة الأحوال", "national id"],
    "مقيم": ["الإقامة", "هوية مقيم", "iqama"],
    "زيارة": ["visit visa", "تمديد الزيارة"],
    "أبشر": ["absher", "منصة ابشر"],
}

def expand_synonyms(q: str) -> List[str]:
    out = [q]
    for key, vals in SYNONYMS.items():
        if key in q:
            out.extend(vals)
    # dedupe
    seen, final = set(), []
    for t in out:
        if t not in seen:
            final.append(t); seen.add(t)
    return final

# ---------------- EN→AR rewriting ----------------
def rewrite_en_to_ar(q: str) -> str:
    try:
        # Use a greedy pipeline for EN->AR rewrite
        gen = pipeline(
            "text-generation",
            model=llm_model,
            tokenizer=llm_tokenizer,
            max_new_tokens=64,
            do_sample=False, # Greedy
            pad_token_id=llm_tokenizer.eos_token_id
        )
        prompt = f"Translate to Arabic (query style):\nEnglish: {q}\nArabic:"
        out = gen(prompt)[0]["generated_text"]
        return normalize_ar(out.split("Arabic:", 1)[-1])
    except:
        return q

def rewrite_and_expand(q: str) -> Tuple[str, List[str]]:
    if looks_english(q):
        q_ar = rewrite_en_to_ar(q)
    else:
        q_ar = normalize_ar(q)
    expanded = expand_synonyms(q_ar.lower())
    return q_ar, expanded

# ---------------- Hybrid Retrieval ----------------
bm25 = BM25Retriever.from_documents(processed_chunks)
bm25.k = 12

dense = vector_store.as_retriever(search_kwargs={"k": 12})

def rrf_merge(a: List[Document], b: List[Document], k=60) -> List[Document]:
    scores, store = {}, {}
    def key(d): return f"{d.metadata.get('service_id','')}::{d.page_content[:120]}"
    for r,d in enumerate(a,1):
        K = key(d); scores[K]=scores.get(K,0)+1/(k+r); store[K]=d
    for r,d in enumerate(b,1):
        K = key(d); scores[K]=scores.get(K,0)+1/(k+r); store[K]=d
    ordered = sorted(scores, key=lambda x: scores[x], reverse=True)
    return [store[k] for k in ordered]

# ---------------- Dense Re-rank ----------------
import numpy as np
def cosine(a,b):
    a=np.asarray(a); b=np.asarray(b)
    denom=(np.linalg.norm(a)*np.linalg.norm(b)) or 1e-9
    return float(np.dot(a,b)/denom)

def dense_rerank(query, docs, top_k=6):
    qv = embedding_model.embed_query(query)
    scored=[]
    for d in docs[:24]:
        dv = embedding_model.embed_query(d.page_content)
        scored.append((cosine(qv,dv),d))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [d for _,d in scored[:top_k]]

# ---------------- Intent classification ----------------
def detect_intent(q: str) -> str:
    nq = normalize_ar(q)
    if "جواز" in nq or "passport" in nq: return "passport"
    if "مقيم" in nq or "اقامة" in nq or "iqama" in nq: return "iqama"
    if "هوية" in nq and "مقيم" not in nq: return "national_id"
    return "generic"

# ---------------- Domain filtering ----------------
def domain_filter(docs: List[Document], intent: str) -> List[Document]:
    out=[]
    for d in docs:
        t = normalize_ar(d.page_content)
        if intent=="passport" and ("اقامة" in t or "مقيم" in t): continue
        if intent=="iqama" and ("هوية وطنية" in t or "الاحوال" in t): continue
        if intent=="national_id" and ("اقامة" in t or "مقيم" in t): continue
        out.append(d)
    return out if out else docs

# ---------------- Language preference (session-level) ----------------
class LangPref:
    def __init__(self): self.pref="auto"
    def update(self, q: str):
        ql = q.lower()
        if any(t in ql for t in ["speak in english","answer in english","english please","in english"]):
            self.pref="en"
        elif any(t in ql for t in ["بالعربي","عربي","باللغة العربية"]):
            self.pref="ar"

    def final(self, q: str):
        if self.pref!="auto": return self.pref
        return "en" if looks_english(q) else "ar"

lang_pref = LangPref()

# ---------------- System Prompt ----------------
SYSTEM_PROMPT = """
<s>[INST] <<SYS>>
أنت مساعد ذكي لخدمات وزارة الداخلية ومنصة أبشر.

السياسات:
- أجب بالعربية إلا إذا طلب المستخدم الإنجليزية أو سأل بها.
- لا تذكر رسوم/شروط/مدد إلا إذا ظهرت صراحة في "السياق".
- إن كان السياق غير كافٍ، قل: "المعلومة غير متوفرة في المستند."
- يسمح بالدردشة الخفيفة بنبرة سعودية لبقة.
- يمنع خلط:
  • الهوية الوطنية (أحوال) ≠ هوية مقيم / الإقامة.
<</SYS>>

[Context]
{context}

[User Question]
{question}
[/INST]
"""

QA_PROMPT = PromptTemplate(
    input_variables=["context","question"],
    template=SYSTEM_PROMPT
)

# ---------------- HF Generation Pipeline ----------------
llm_pipe = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=llm_model,
        tokenizer=llm_tokenizer,
        max_new_tokens=700,
        temperature=0.55,
        top_p=0.92,
        do_sample=True,
        repetition_penalty=1.10,
        pad_token_id=llm_tokenizer.eos_token_id
    )
)

def clean_out(txt): return txt.split("[/INST]",1)[-1].strip()

# ---------------- ⭐️ NEW Translation Pipeline ----------------
# We create a dedicated, non-sampling (greedy) pipeline for translation
try:
    translation_pipeline = pipeline(
        "text-generation",
        model=llm_model,
        tokenizer=llm_tokenizer,
        max_new_tokens=1024, # Allow longer translations
        do_sample=False,    # MUST be False for accurate translation
        pad_token_id=llm_tokenizer.eos_token_id
    )
except Exception as e:
    print(f"⚠️ Warning: Could not create translation pipeline: {e}")
    translation_pipeline = llm_pipe # Fallback to main pipe

def post_hoc_translate(text_to_translate: str, target_lang: str) -> str:
    """Translate text to the target language using the greedy LLM pipeline."""
    
    if target_lang == "en":
        prompt = f"Translate this Arabic text to English. Provide *only* the translated text.\nArabic: {text_to_translate}\nEnglish:"
    else:
        prompt = f"Translate this English text to Arabic. Provide *only* the translated text.\nEnglish: {text_to_translate}\nArabic:"

    try:
        raw_trans = translation_pipeline(prompt)[0]["generated_text"]
        
        if target_lang == "en":
            cleaned = raw_trans.split("English:", 1)[-1].strip()
        else:
            cleaned = raw_trans.split("Arabic:", 1)[-1].strip()
        
        # Fallback cleaning if the model includes the prompt
        if not cleaned or "translate this" in cleaned.lower():
             cleaned = raw_trans.splitlines()[-1].strip()

        return cleaned if cleaned else text_to_translate

    except Exception as e:
        logging.warning(f"Post-hoc translation failed: {e}")
        return text_to_translate # Return original on error

# ---------------- Main RAG Class (Updated) ----------------
class ProRAGChain:
    def __init__(self): self.last_sources=[]

    def _ctx(self, docs):
        blocks=[]
        for d in docs[:6]:
            title = d.metadata.get("chunk_title") or d.metadata.get("service_title")
            sid   = d.metadata.get("service_id","")
            blocks.append(f"(service_id={sid}) {title}\n{d.page_content}")
        return "\n\n---\n\n".join(blocks)

    def invoke(self, inputs):
        q = inputs.get("question","")

        # 1. Update language preference
        lang_pref.update(q)
        final_lang = lang_pref.final(q)

        # 2. Rewrite + expand
        q_ar, expanded = rewrite_and_expand(q)

        # 3. Retrieval
        dense_docs = dense.get_relevant_documents(q_ar)
        bm_docs    = bm25.get_relevant_documents(" ".join(expanded))
        merged     = rrf_merge(dense_docs, bm_docs)

        # 4. Intent + domain filtering
        intent = detect_intent(q)
        merged = domain_filter(merged, intent)

        # 5. Re-rank
        final_docs = dense_rerank(q_ar, merged, top_k=6)
        self.last_sources = final_docs

        # 6. Generate Answer
        lang_tag = "Answer in English." if final_lang=="en" else "أجب بالعربية."
        prompt = QA_PROMPT.format(
            context=self._ctx(final_docs),
            question=f"{q}\n\n{lang_tag}"
        )

        raw = llm_pipe.invoke(prompt)
        ans = clean_out(raw)

        # 7. ⭐️ NEW: Post-hoc Translation Check ⭐️
        ans_is_arabic = is_arabic(ans)
        
        if final_lang == "en" and ans_is_arabic:
            logging.info("Post-hoc translation required: AR -> EN")
            ans = post_hoc_translate(ans, "en")
        
        elif final_lang == "ar" and not ans_is_arabic and looks_english(ans):
            # Also handle the reverse case (e.g., chitchat in EN)
            logging.info("Post-hoc translation required: EN -> AR")
            ans = post_hoc_translate(ans, "ar")

        return {"answer": ans}

qa_chain = ProRAGChain()
print("✅ PRO RAG Chain (v2) is ready.")

Device set to use cuda:0
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧠 Building PRO RAG chain (v2 — Post-hoc Translation)…
✅ PRO RAG Chain (v2) is ready.


In [5]:
# @title 🤖 CELL 5 — Pro Chat UI (MOI Edition - Gradio 3.50 Stable) 🇸🇦
# Replaces ipywidgets with a modern, multi-page Gradio Web App.

import gradio as gr
import time
import os

# Disable tokenizers parallelism to avoid deadlocks
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("🚀 Launching MOI Branded Interface...")

# =====================================================
# 1. Custom CSS (MOI Theme)
# =====================================================
moi_css = """
.gradio-container { background-color: #f4f6f8 !important; }
.moi-header {
    text-align: center;
    padding: 30px;
    background: linear-gradient(90deg, #006C35 0%, #004D26 100%);
    border-radius: 10px;
    color: white;
    margin-bottom: 20px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.2);
}
.moi-header h1 { color: white !important; font-size: 2.5em; margin: 0; }
.moi-header p { color: #d4af37 !important; font-size: 1.2em; margin-top: 5px; }
.lang-btn { font-size: 1.2em; height: 60px; border-radius: 8px !important; }
"""

# =====================================================
# 2. Logic Handlers
# =====================================================
def start_chat(lang):
    """Transition from Welcome Screen to Chat Screen"""
    # Configure UI based on language choice
    if lang == "Arabic":
        greeting = [(None, "👋 حياك الله! أنا مساعدك الذكي لخدمات وزارة الداخلية. تفضل بطرح سؤالك.")]
        rtl = True
        label = "المحادثة الفورية"
        placeholder = "اكتب سؤالك هنا..."
    else:
        greeting = [(None, "👋 Hello! I am your MOI Smart Assistant. How can I help you today?")]
        rtl = False
        label = "Live Chat"
        placeholder = "Type your question here..."
        
    return (
        gr.update(visible=False),           # Hide Welcome Screen
        gr.update(visible=True),            # Show Chat Screen
        greeting,                           # Set History
        lang,                               # Set State
        gr.update(label=label, rtl=rtl),    # Update Chatbot Label/Dir
        gr.update(placeholder=placeholder, rtl=rtl) # Update Input Dir
    )

def chat_response(message, history, audio_file, lang_val):
    """Handles text/audio input and calls the backend chain."""
    
    # 1. Handle Audio (Priority)
    if audio_file:
        try:
            target_lang = "ar" if lang_val == "Arabic" else "en"
            # Check if Whisper is loaded
            if 'asr_pipe' in globals() and asr_pipe:
                text = asr_pipe(audio_file, generate_kwargs={"language": target_lang})["text"].strip()
                message = text
                user_display = f"🎤 {text}"
            else:
                return history + [[None, "⚠️ Error: Whisper model not loaded."]]
        except Exception as e:
            return history + [[None, f"❌ Audio Error: {str(e)}"]]
    else:
        user_display = message

    # 2. Validate Input
    if not message: return history

    # 3. Call the Brain (Hybrid Chain)
    try:
        # Support both naming conventions (Legacy vs Golden)
        if 'hybrid_chain' in globals():
            response = hybrid_chain.answer(message)
        elif 'qa_chain' in globals():
            # Fallback to legacy chain if hybrid isn't ready
            res = qa_chain.invoke({"question": message})
            response = res.get("answer", "")
        else:
            response = "⚠️ System Warning: AI Brain not loaded. Please run previous cells."
            
    except Exception as e:
        response = f"❌ System Error: {e}"

    # 4. Update History
    history.append((user_display, response))
    return history

def reset_app():
    """Return to language selection"""
    return (
        gr.update(visible=True),  # Show Welcome
        gr.update(visible=False), # Hide Chat
        None,                     # Clear State
        []                        # Clear History
    )

def clear_inputs(): return "", None

# =====================================================
# 3. Layout (Blocks)
# =====================================================
with gr.Blocks(theme=gr.themes.Soft(), css=moi_css, title="MOI Assistant") as demo:
    
    # State to hold language preference
    lang_state = gr.State(value="Arabic")

    # --- Header ---
    gr.HTML("""
    <div class='moi-header'>
        <h1>المساعد الذكي لوزارة الداخلية</h1>
        <p>MOI Smart Assistant | Powered by ALLaM-7B</p>
    </div>
    """)
    
    # --- SCREEN 1: Welcome & Language Selection ---
    with gr.Group(visible=True) as welcome_screen:
        gr.Markdown(
            "<h3 style='text-align: center;'>🌍 Please select your preferred language / الرجاء اختيار اللغة المفضلة</h3>"
        )
        with gr.Row():
            btn_ar = gr.Button("العربية 🇸🇦", variant="primary", elem_classes=["lang-btn"])
            btn_en = gr.Button("English 🇬🇧", variant="secondary", elem_classes=["lang-btn"])

    # --- SCREEN 2: Chat Interface ---
    with gr.Group(visible=False) as chat_screen:
        with gr.Row():
            # Left: Chatbot Window
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(label="Chat", height=500)
                with gr.Row():
                    msg = gr.Textbox(show_label=False, container=False, scale=4)
                    submit_btn = gr.Button("🚀", variant="primary", scale=1)

            # Right: Settings & Audio
            with gr.Column(scale=1):
                gr.Markdown("### ⚙️ أدوات / Tools")
                
                # Audio Input
                audio_input = gr.Audio(source="microphone", type="filepath", label="تسجيل صوتي / Voice Input")
                
                gr.Markdown("---")
                restart_btn = gr.Button("🔄 تغيير اللغة / Change Language", variant="secondary")

    # =====================================================
    # 4. Event Wiring
    # =====================================================
    
    # Language Selection Events
    btn_ar.click(
        fn=lambda: start_chat("Arabic"),
        outputs=[welcome_screen, chat_screen, chatbot, lang_state, chatbot, msg]
    )
    btn_en.click(
        fn=lambda: start_chat("English"),
        outputs=[welcome_screen, chat_screen, chatbot, lang_state, chatbot, msg]
    )

    # Chat Events (Text & Audio)
    msg.submit(chat_response, [msg, chatbot, audio_input, lang_state], [chatbot]) \
       .then(clear_inputs, None, [msg, audio_input])
    
    submit_btn.click(chat_response, [msg, chatbot, audio_input, lang_state], [chatbot]) \
              .then(clear_inputs, None, [msg, audio_input])

    # Restart Event
    restart_btn.click(reset_app, outputs=[welcome_screen, chat_screen, lang_state, chatbot])

# =====================================================
# 5. Launch
# =====================================================
print("✅ Launching MOI App...")
# share=True creates a public link for mobile testing
demo.queue().launch(share=True, inline=False, show_api=False)

🚀 Launching MOI Branded Interface...
✅ Launching MOI App...
Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://a254b64afd33279a25.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
